In [99]:
import pandas as pd
from docx import Document

df = pd.read_parquet("data.parquet")

In [100]:

def replace_text(doc, replacements):
    # Función para reemplazar texto en runs sin perder formato y sin cortar texto
    def replace_in_paragraph(paragraph, replacements):
        # Concatenamos el texto completo de todos los runs
        full_text = ''.join(run.text for run in paragraph.runs)
        
        # Realizamos los reemplazos en el texto completo
        for old_text, new_text in replacements.items():
            if old_text in full_text:
                full_text = full_text.replace(old_text, new_text)
        
        # Limpiamos todos los runs del párrafo
        for run in paragraph.runs:
            run.clear()

        # Volvemos a agregar el texto completo en el primer run para mantener el formato del párrafo
        paragraph.add_run(full_text)

    # Reemplazar texto en los párrafos del documento
    for paragraph in doc.paragraphs:
        replace_in_paragraph(paragraph, replacements)

    # Reemplazar texto dentro de las tablas
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    replace_in_paragraph(paragraph, replacements)


In [101]:
def convertir_fecha(valor):
    # Convierte el valor a cadena para separar año y mes
    valor_str = str(valor)
    año = valor_str[:4]
    mes_num = valor_str[4:6]
    
    # Diccionario para mapear el número del mes al nombre en español
    meses = {
        '01': 'Enero', '02': 'Febrero', '03': 'Marzo', '04': 'Abril',
        '05': 'Mayo', '06': 'Junio', '07': 'Julio', '08': 'Agosto',
        '09': 'Septiembre', '10': 'Octubre', '11': 'Noviembre', '12': 'Diciembre'
    }
    
    # Obtiene el nombre del mes
    mes_nombre = meses.get(mes_num, 'Mes inválido')
    
    # Formatea el resultado
    return f"{año} - {mes_nombre}"

In [102]:
doc = Document('plantilla.docx')

In [103]:
def process_text(busqueda):
    # Abre el documento de Word
    try:
        row = df[df['ID_EMPRESA']==busqueda]
    except:
        return print('No se encontró la empresa')
    
    print(row['V_NUMPERIOD'].values[0])
    Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7 = convertir_fecha(row['V_NUMPERIOD'].values[0]), row['ID_EMPRESA'], row['sector'], row['Departamento_ct'], row['anio_anti'],row['numtra'], row['costosal'], row['sst']
    
    # print(convertir_fecha(row['V_NUMPERIOD'].values[0]))
    
    # Variables A,B
    A0, A1, A2, A3, A4, A5 = row['ntrab_rl_privgen'], row['ntrab_rl_micro'], row['ntrab_rl_peq'], row['ntrab_rl_agrar'], row['ntrab_rl_otros'], row['ntrab_rl_noprec']
    suma_A = A0 + A1 + A2 + A3 + A4 + A5
    B0, B1, B2, B3, B4, B5 = [round(round(val / suma_A, 4) * 100,2) for val in (A0, A1, A2, A3, A4, A5)]

    # Variables C,D
    C0, C1, C2, C3, C4, C5, C6 = row['ntrab_tc_indet'], row['ntrab_tc_nattemp'], row['ntrab_tc_natacc'], row['ntrab_tc_obrserv'], row['ntrab_tc_tiempar'], row['ntrab_tc_otros'], row['ntrab_tc_noprec']
    suma_C = C0 + C1 + C2 + C3 + C4 + C5 + C6
    D0, D1, D2, D3, D4, D5, D6 = [round(round(val / suma_C, 4) * 100,2) for val in (C0, C1, C2, C3, C4, C5, C6)]

    # Variables E,F
    E0,E1,E2,E3 = row['ntrab_rp_spp'], row['ntrab_rp_snp'], row['ntrab_rp_otros'], row['ntrab_rp_noprecrp']
    suma_E = E0 + E1 + E2 + E3
    F0,F1,F2,F3 = [round(round(val / suma_C, 4) * 100,2) for val in (E0,E1,E2,E3)]

    # Variables G,H
    G0,G1,G2,G3 = row['ntrab_afil_essalud'], row['ntrab_afil_essalud_agrar'], row['ntrab_afil_sis'], row['ntrab_afil_noprec']
    suma_G = G0 + G1 + G2 + G3
    H0, H1, H2, H3 = [round(round(val / suma_G, 4) * 100,2) for val in (G0,G1,G2,G3)]

    # Variables I,J
    I0,I1,I2 = row['ntrab_18mas'], row['ntrab_menos18'], row['ntrab_edad_noprec']
    suma_I = I0 + I1 + I2
    J0, J1, J2 = [round(round(val / suma_I, 4) * 100,2) for val in (I0,I1,I2)]

    # Variables K,L
    K0,K1,K2 = row['ntrab_hombres'], row['ntrab_mujeres'], row['ntrab_sexo_noprec']
    suma_K = K0 + K1 + K2
    L0, L1, L2 = [round(round(val / suma_K, 4) * 100,2)for val in (K0, K1, K2)]

    # Variables M,N
    M0, M1 = row['ntrab_nosind'], row['ntrab_sind']
    suma_M = M0 + M1
    N0, N1 = [round(round(val / suma_M, 4) * 100,2) for val in (M0, M1)]

    # Variables O,P
    O0, O1, O2 = row['ntrab_consctr'], row['ntrab_sinsctr'], row['ntrab_sexo_noprec']
    suma_O = O0 + O1 + O2
    P0, P1, P2 = [round(round(val / suma_O, 4) * 100,2) for val in (O0, O1, O2)]
    
    # Diccionario de reemplazos
    reemplazos = {
        'Z0': str(Z0),
        'Z1': str(Z1.values[0]),
        'Z2': str(Z2.values[0]),
        'Z3': str(Z3.values[0]),
        'Z4': str(Z4.values[0]),
        'Z5': str(Z5.values[0]),
        'Z6': str(Z6.values[0]),
        'Z7': str(Z7.values[0]),
        'A0': str(A0.values[0]),
        'A1': str(A1.values[0]),
        'A2': str(A2.values[0]),
        'A3': str(A3.values[0]),
        'A4': str(A4.values[0]),
        'A5': str(A5.values[0]),
        'B0': str(B0.values[0]), 
        'B1': str(B1.values[0]), 
        'B2': str(B2.values[0]), 
        'B3': str(B3.values[0]), 
        'B4': str(B4.values[0]), 
        'B5': str(B5.values[0]),
        'C0': str(C0.values[0]),
        'C1': str(C1.values[0]),
        'C2': str(C2.values[0]),
        'C3': str(C3.values[0]),
        'C4': str(C4.values[0]),
        'C5': str(C5.values[0]),
        'C6': str(C6.values[0]),
        'D0': str(D0.values[0]),
        'D1': str(D1.values[0]),
        'D2': str(D2.values[0]),
        'D3': str(D3.values[0]),
        'D4': str(D4.values[0]),
        'D5': str(D5.values[0]),
        'D6': str(D6.values[0]),
        'E0': str(E0.values[0]),
        'E1': str(E1.values[0]),
        'E2': str(E2.values[0]),
        'E3': str(E3.values[0]),
        'F0': str(F0.values[0]),
        'F1': str(F1.values[0]),
        'F2': str(F2.values[0]),
        'F3': str(F3.values[0]),
        'G0': str(G0.values[0]),
        'G1': str(G1.values[0]),
        'G2': str(G2.values[0]),
        'G3': str(G3.values[0]),
        'H0': str(H0.values[0]),
        'H1': str(H1.values[0]),
        'H2': str(H2.values[0]),
        'H3': str(H3.values[0]),
        'I0': str(I0.values[0]),
        'I1': str(I1.values[0]),
        'I2': str(I2.values[0]),
        'J0': str(J0.values[0]),
        'J1': str(J1.values[0]),
        'J2': str(J2.values[0]),
        'K0': str(K0.values[0]),
        'K1': str(K1.values[0]),
        'K2': str(K2.values[0]),
        'L0': str(L0.values[0]),
        'L1': str(L1.values[0]),
        'L2': str(L2.values[0]),
        'M0': str(M0.values[0]),
        'M1': str(M1.values[0]),
        'N0': str(N0.values[0]),
        'N1': str(N1.values[0]),
        'O0': str(O0.values[0]),
        'O1': str(O1.values[0]),
        'O2': str(O2.values[0]),
        'P0': str(P0.values[0]),
        'P1': str(P1.values[0]),
        'P2': str(P2.values[0])
    }
    
    replace_text(doc, reemplazos)
    
    doc.save(f"Planilla_Electrónica_{row['ID_EMPRESA'].values[0]}.docx")
    print('Documento Generado')
    return row['ID_EMPRESA'].values[0]

In [104]:
id = process_text('6UV9S7A9Q3E')

202404
Documento Generado


In [105]:
from docx.shared import Pt  # Para el tamaño de fuente

# Función para aplicar formato a una celda completa
def apply_format_to_cell(cell, bold=False, font_size=None):
    for paragraph in cell.paragraphs:
        for run in paragraph.runs:
            # Aplicar formato de negrita si se especifica
            if bold:
                run.bold = True
            # Aplicar tamaño de fuente si se especifica
            if font_size:
                run.font.size = Pt(font_size)

# Función para aplicar formato a un texto específico en el documento
def apply_format_to_text(doc, text_to_format, bold=False, font_size=None):
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            if text_to_format in run.text:
                # Aplicar formato de negrita si se especifica
                if bold:
                    run.bold = True
                # Aplicar tamaño de fuente si se especifica
                if font_size:
                    run.font.size = Pt(font_size)

    # Formato en tablas
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        if text_to_format in run.text:
                            if bold:
                                run.bold = True
                            if font_size:
                                run.font.size = Pt(font_size)


# Aplicar formato a un texto específico en todo el documento
apply_format_to_text(doc, "Planilla Electrónica", bold=False, font_size=18)

apply_format_to_text(doc, "Empresa:", bold=True, font_size=12)
apply_format_to_text(doc, "Departamento", bold=True, font_size=12)
apply_format_to_text(doc, "Sector", bold=True, font_size=12)
apply_format_to_text(doc, "Años en el mercado", bold=True, font_size=12)
apply_format_to_text(doc, "Costo Salarial", bold=True, font_size=12)
apply_format_to_text(doc, "SST", bold=True, font_size=12)


apply_format_to_text(doc, "Nº de trabajadores", bold=True, font_size=12)

apply_format_to_text(doc, "Régimen Laboral", bold=True, font_size=11)
apply_format_to_text(doc, "Régimen Pensionario", bold=True, font_size=11)
apply_format_to_text(doc, "Régimen Laboral", bold=True, font_size=11)
apply_format_to_text(doc, "Afiliación Sindical", bold=True, font_size=11)
apply_format_to_text(doc, "Rango de Edades", bold=True, font_size=11)
apply_format_to_text(doc, "Tipo de Contrato", bold=True, font_size=11)
apply_format_to_text(doc, "Régimen de Salud", bold=True, font_size=11)
apply_format_to_text(doc, "Género", bold=True, font_size=11)
apply_format_to_text(doc, "Afiliación al SCTR", bold=True, font_size=11)
apply_format_to_text(doc, "N° Trabj.", bold=True, font_size=11)
apply_format_to_text(doc, "% Trabj.", bold=True, font_size=11)


doc.save(f"Planilla_Electrónica_{id}.docx")